Gender Biases in Student Evaluations of Teachers - A Randomized, Online Experiment
====================================================


In [6]:
# boilerplate
%matplotlib inline
import math
import numpy as np
import pandas as pd
from numpy.random import random
import scipy as sp
import matplotlib.pyplot as plt
from __future__ import division
from scipy.stats import ttest_ind

# initialize PRNG
rs = np.random.RandomState(seed=1)

Permutation test code
============
You must install the _permute_ package to use this code. Install instructions can be found at https://github.com/statlab/permute.

In [7]:
from permute.core import corr, two_sample
from permute.utils import get_prng, permute_within_groups

def stratified_corr(x, y, group, alternative="greater", reps=10**5, keep_dist=False, seed=None):
    """
    Tests the null hypothesis of 0 correlation between x and y, against the alternative
    hypothesis that the correlation is
    (a) greater than 0 if side = 'greater'
    (b) less than 0 if side = 'less'
    (c) different from 0 if side = 'two-sided'

    Permutations are carried out within the given groups.  Under the null hypothesis,
    observations within each group are exchangeable.

    If ``keep_dist``, return the distribution of values of the test statistic;
    otherwise, return only the number of permutations for which the value of
    the test statistic and p-value.

    Parameters
    ----------
    x : array-like
        First variable
    y : array-like
        Second variable
    group : array-like
        Group assignments; permutations are done within each group
    alternative : {'greater', 'less', 'two-sided'}
        The alternative hypothesis to test
    reps : int
        Number of permutations
    keep_dist : bool
        flag for whether to store and return the array of values
        of the test statistic
    seed : RandomState instance or {None, int, RandomState instance}
        If None, the pseudorandom number generator is the RandomState
        instance used by `np.random`;
        If int, seed is the seed used by the random number generator;
        If RandomState instance, seed is the pseudorandom number generator.

    Returns
    -------
    float
        the estimated p-value
    float
        the test statistic
    list
        The distribution of test statistics.
        These values are only returned if `keep_dist` == True        
    """
    
    prng = get_prng(seed)
    
    observed_tst = np.ma.corrcoef(np.ma.array(x, mask=np.isnan(x)), np.ma.array(y, mask=np.isnan(y)))[0,1]
    theStat = {
        'greater': lambda u,v: np.ma.corrcoef(np.ma.array(u, mask=np.isnan(u)), np.ma.array(v, mask=np.isnan(v)))[0,1],
        'less': lambda u,v: -np.ma.corrcoef(np.ma.array(u, mask=np.isnan(u)), np.ma.array(v, mask=np.isnan(v)))[0,1],
        'two-sided': lambda u,v: math.fabs(np.ma.corrcoef(np.ma.array(u, mask=np.isnan(u)), np.ma.array(v, mask=np.isnan(v)))[0,1])
    }
    tst = theStat[alternative](x, y)
    
    if keep_dist:
        dist = np.empty(reps)
        for i in range(reps):
            dist[i] = theStat[alternative](x, permute_within_groups(y, group, seed=prng))
        hits = np.sum(dist >= tst)
        return hits/reps, observed_tst, dist
    else:
        hits = np.sum([(theStat[alternative](x, permute_within_groups(y, group, seed=prng)) >= tst)
                       for i in range(reps)])
        return hits/reps, observed_tst
    
    
def stratified_two_sample(x, y, group_x, group_y, stat='mean', alternative="greater", reps=10**5, 
                          keep_dist=False, seed=None):
    """
    One-sided or two-sided, two-sample permutation test for equality of
    two means, with p-value estimated by simulated random sampling with
    reps replications.

    Tests the hypothesis that x and y are a random partition of x,y
    against the alternative that x comes from a population with mean

    (a) greater than that of the population from which y comes,
        if side = 'greater'
    (b) less than that of the population from which y comes,
        if side = 'less'
    (c) different from that of the population from which y comes,
        if side = 'two-sided'

    Permutations are carried out within the given groups.  Under the null hypothesis,
    observations within each group are exchangeable.

    If ``keep_dist``, return the distribution of values of the test statistic;
    otherwise, return only the number of permutations for which the value of
    the test statistic and p-value.

    Parameters
    ----------
    x : array-like
        Sample 1
    y : array-like
        Sample 2
    group_x : array-like
        Group assignments for sample 1
    group_y : array-like
        Group assignments for sample 2
    stat : {'mean', 't'}
        The test statistic.

        (a) If stat == 'mean', the test statistic is (mean(x) - mean(y))
            (equivalently, sum(x), since those are monotonically related), omitting
            NaNs, which therefore can be used to code non-responders
        (b) If stat == 't', the test statistic is the two-sample t-statistic--
            but the p-value is still estimated by the randomization,
            approximating the permutation distribution.
            The t-statistic is computed using scipy.stats.ttest_ind
        (c) If stat is a function (a callable object), the test statistic is
            that function.  The function should take a permutation of the pooled
            data and compute the test function from it. For instance, if the
            test statistic is the Kolmogorov-Smirnov distance between the
            empirical distributions of the two samples, max_t |F_x(t) - F_y(t)|,
            the test statistic could be written:

            f = lambda u: np.max( \
                [abs(sum(u[:len(x)]<=v)/len(x)-sum(u[len(x):]<=v)/len(y)) for v in u]\
                )        
    alternative : {'greater', 'less', 'two-sided'}
        The alternative hypothesis to test
    reps : int
        Number of permutations
    keep_dist : bool
        flag for whether to store and return the array of values
        of the test statistic
    seed : RandomState instance or {None, int, RandomState instance}
        If None, the pseudorandom number generator is the RandomState
        instance used by `np.random`;
        If int, seed is the seed used by the random number generator;
        If RandomState instance, seed is the pseudorandom number generator.

    Returns
    -------
    float
        the estimated p-value
    float
        the test statistic
    list
        The distribution of test statistics.
        These values are only returned if `keep_dist` == True        
    """
    
    prng = get_prng(seed)
    
    z = np.concatenate([x, y])   # pooled responses
    groups = np.concatenate([group_x, group_y])   # pooled group assignments
    
    # If stat is callable, use it as the test function. Otherwise, look in the dictionary
    stats = {
        'mean': lambda u: np.nanmean(u[:len(x)]) - np.nanmean(u[len(x):]),
        't': lambda u: ttest_ind(
            u[:len(x)][~np.isnan(u[:len(x)])], 
            u[len(x):][~np.isnan(u[len(x):])], 
            equal_var=True)[0]
    }
    if callable(stat):
        tst_fun = stat
    else:
        tst_fun = stats[stat]

    theStat = {
        'greater': tst_fun,
        'less': lambda u: -tst_fun(u),
        'two-sided': lambda u: math.fabs(tst_fun(u))
    }
    tst = theStat[alternative](z)
    observed_tst = tst_fun(z)
    
    if keep_dist:
        dist = np.empty(reps)
        for i in range(reps):
            dist[i] = theStat[alternative](permute_within_groups(z, groups, seed=prng))
        hits = np.sum(dist >= tst)
        return hits/reps, tst, dist
    else:
        hits = np.sum([(theStat[alternative](permute_within_groups(z, groups, seed=prng)) >= tst)
                       for i in range(reps)])
        return hits/reps, tst

Read data
=================

Some notes on the variables:
* **group** identifies the section the student was placed in.
* **gender** refers to the student's gender: 1 = male, 2 = female.
* **tagender** is the instructor's true gender: 1 = male, 0 = female.
* **taidgender** is the instructor's reported gender: 1 = male, 0 = female.
* **grade** is on a scale from 0-100

The IRB did not allow grades to be linked to ratings. There are 43 ratings and 47 grades. Four students did not submit evaluations (3 in group 5 and 1 in group 6), but we do not know how each rated his or her instructor. 

In [8]:
ratings = pd.read_csv("Macnell-RatingsData.csv")
categories = ratings.columns.values.tolist()[1:15]
print ratings.head()
print ratings.describe()

   group  professional  respect  caring  enthusiastic  communicate  helpful  \
0      3             5        5       4             4            4        3   
1      3             4        4       4             4            5        5   
2      3             5        5       5             5            5        5   
3      3             5        5       5             5            5        3   
4      3             5        5       5             5            5        5   

   feedback  prompt  consistent  fair  responsive  praised  knowledgeable  \
0         4       4           4     4           4        4              3   
1         5       5           3     4           5        5              5   
2         5       5           5     5           5        5              5   
3         5       5           5     5           3        5              5   
4         5       3           4     5           5        5              5   

   clear  overall  gender   age  tagender  taidgender  
0     

In [9]:
grades = pd.read_csv("Macnell-GradeData.csv")
print grades.head()
print grades.describe()

   group  grade  tagender  taidgender
0      3  77.40         0           1
1      3  89.02         0           1
2      3  53.50         0           1
3      3  88.32         0           1
4      3  90.02         0           1
           group      grade   tagender  taidgender
count  47.000000  47.000000  47.000000   47.000000
mean    4.531915  79.008936   0.510638    0.489362
std     1.158167   9.971515   0.505291    0.505291
min     3.000000  49.460000   0.000000    0.000000
25%     3.500000  75.200000   0.000000    0.000000
50%     5.000000  80.130000   1.000000    0.000000
75%     6.000000  85.095000   1.000000    1.000000
max     6.000000  95.100000   1.000000    1.000000


# Analysis

## Evidence of gender bias

### Ratings vs reported instructor gender (correlation)
We use a stratified test to assess whether the correlation between ratings and reported instructor gender is statistically significant.  The strata are defined by instructor.  Permutations are done within strata (under the null, the rating doesn't depend on reported gender, but may differ by teacher), then strata are pooled to compute an overall Spearman correlation. 
Nonresponders are also assigned at random within strata, but are omitted from the calculation of the mean or correlation.

In [10]:
(p, t) = stratified_corr(ratings['overall'], ratings['taidgender'], ratings['tagender'], alternative = "two-sided")
print 'Overall rating:'
print 'Correlation:', t
print 'P-value (two-sided):', np.round(p, 5), "\n"

print ('\n\n{0:24} {1:8} {2:8}'.format('Category', 'Corr', 'P-value'))
for col in categories:
    (p, t) = stratified_corr(ratings[col], ratings['taidgender'], ratings['tagender'], alternative = "two-sided")
    print ('{0:20} {1:8.2f} {2:8.2f}'.format(col, t, p))

Overall rating:
Correlation: 0.233924970178
P-value (two-sided): 0.11365 



Category                 Corr     P-value 
professional             0.29     0.06
respect                  0.29     0.06
caring                   0.25     0.10
enthusiastic             0.28     0.06
communicate              0.27     0.07
helpful                  0.22     0.17
feedback                 0.21     0.15
prompt                   0.37     0.01
consistent               0.20     0.21
fair                     0.39     0.01
responsive               0.11     0.46
praised                  0.36     0.01
knowledgeable            0.18     0.29
clear                    0.17     0.29


### Ratings vs reported instructor gender (difference in means)
We use a stratified test to evaluate the significance of the difference in means between ratings and reported instructor gender.  As above, strata are defined by actual instructor.  There is a monotone transformation from the correlation to the difference in means statistic, so the two tests are equivalent.  
P-values for these two tests should be the same, up to sampling variability.

In [11]:
(p, t) = stratified_two_sample(ratings['overall'][ratings.taidgender==1], ratings['overall'][ratings.taidgender==0], 
                               ratings['tagender'][ratings.taidgender == 1], 
                               ratings['tagender'][ratings.taidgender == 0],
                               alternative = "two-sided", stat='t', seed = rs)
print 'Overall rating:'
print 'Difference in means:', t
print 'P-value (two-sided):', np.round(p, 5), "\n"

print ('\n\n{0:24} {1:8} {2:8}'.format('Category', 'Diff means', 'P-value'))
for col in categories:
    (p, t) = stratified_two_sample(ratings[col][ratings.taidgender==1], ratings[col][ratings.taidgender==0], 
                               ratings['tagender'][ratings.taidgender == 1],
                               ratings['tagender'][ratings.taidgender == 0],
                               alternative = "two-sided", stat='t', seed = rs)
    print ('{0:20} {1:8.2f} {2:8.2f}'.format(col, t, p))

Overall rating:
Difference in means: 1.54059499043
P-value (two-sided): 0.11173 



Category                 Diff means P-value 
professional             1.93     0.06
respect                  1.93     0.06
caring                   1.66     0.10
enthusiastic             1.90     0.06
communicate              1.78     0.07
helpful                  1.41     0.17
feedback                 1.38     0.15
prompt                   2.57     0.01
consistent               1.30     0.21
fair                     2.68     0.01
responsive               0.72     0.48
praised                  2.48     0.02
knowledgeable            1.18     0.29
clear                    1.07     0.29


### Ratings vs concordance of student and reported instructor genders

We tested the correlation between ratings and gender concordance (1 if the student had the same gender as the reported instructor gender; 0 otherwise). We report the results separately for male students and female students.

In [12]:
ratings['gender_concordance'] = ( (ratings['gender']% 2)==ratings['taidgender'] )
stu_male = ratings[ratings['gender']==1]
stu_female = ratings[ratings['gender']==2]

print 'Male students\n'
print 'Number of male students:', stu_male.shape[0], '\n'
print ('\n{0:15} {1:8} {2:8}'.format('Category', 'Correlation','Two-sided p-value'))
(p, t) = stratified_corr(x = stu_male['overall'], y = stu_male['gender_concordance'],
                         group = stu_male['tagender'], alternative="two-sided", seed = rs)
print ('{0:15} {1:8.3f} {2:10.2f}'.format('Overall', t, p))

for col in categories:
    (p, t) = stratified_corr(x = stu_male[col], y = stu_male['gender_concordance'], 
                  group = stu_male['tagender'], alternative="two-sided", seed = rs)
    print ('{0:15} {1:8.3f} {2:10.2f}'.format(col, t, p))

print 'Female students\n'
print 'Number of female students:', stu_female.shape[0], '\n'
print ('\n{0:15} {1:8} {2:8}'.format('Category', 'Correlation','Two-sided p-value'))
(p, t) = stratified_corr(x = stu_female['overall'], y = stu_female['gender_concordance'], 
                         group = stu_female['tagender'], alternative="two-sided", seed = rs)
print ('{0:15} {1:8.3f} {2:10.2f}'.format('Overall', t, p))

for col in categories:
    (p, t) = stratified_corr(x = stu_female[col], y = stu_female['gender_concordance'], 
                  group = stu_female['tagender'], alternative="two-sided", seed = rs)
    print ('{0:15} {1:8.3f} {2:10.2f}'.format(col, t, p))

Male students

Number of male students: 20 


Category        Correlation Two-sided p-value
Overall            0.090       0.81
professional       0.217       0.40
respect            0.217       0.33
caring             0.020       1.00
enthusiastic       0.090       0.82
communicate        0.123       0.67
helpful            0.211       0.42
feedback           0.040       0.90
prompt             0.377       0.15
consistent         0.073       0.84
fair               0.408       0.09
responsive         0.181       0.53
praised            0.287       0.26
knowledgeable      0.078       0.77
clear              0.056       0.76
Female students

Number of female students: 23 


Category        Correlation Two-sided p-value
Overall           -0.364       0.11
professional      -0.361       0.09
respect           -0.361       0.09
caring            -0.458       0.05
enthusiastic      -0.440       0.05
communicate       -0.394       0.10
helpful           -0.242       0.35
feedback          -0

### Ratings vs concordance of student and actual instructor genders

Since the students didn't know the instructors' actual gender, we hope that there is no correlation between gender concordance and ratings. This analysis is unstratified.  We report the results separately for male students and female students.

In [13]:
ratings['gender_concordance_actual'] = ( (ratings['gender']% 2)==ratings['tagender'] )
stu_male = ratings[ratings['gender']==1]
stu_female = ratings[ratings['gender']==2]

(t, plow, pupper, pboth, sims) = corr(x = stu_male['overall'], \
                                      y = stu_male['gender_concordance_actual'], seed = rs)
print 'Male students\n'
print 'Number of male students:', stu_male.shape[0]
print 'Correlation:', t
print 'Upper p-value:', pupper
print 'Two-sided p-value:', pboth
print ('\n{0:15} {1:8} {2:8} {3:8}'.format('Category', 'Correlation',\
                                           'Upper p-value', 'Two-sided p-value'))
(t, plow, pupper, pboth, sims) = corr(x = stu_male['overall'], \
                                      y = stu_male['gender_concordance_actual'], seed = rs)
print ('{0:15} {1:8.3f} {2:10.2f} {3:10.2f}'.format('Overall', t, pupper, pboth))

for col in categories:
    (t, plow, pupper, pboth, sims) = corr(x = stu_male[col], \
                                          y = stu_male['gender_concordance_actual'], seed = rs)
    print ('{0:15} {1:8.3f} {2:10.2f} {3:10.2f}'.format(col, t, pupper, pboth))


(t, plow, pupper, pboth, sims) = corr(x = stu_female['overall'], \
                                      y = stu_female['gender_concordance_actual'], seed = rs)
print '\nFemale students\n'
print 'Number of female students:', stu_female.shape[0]
print 'Correlation:', t
print 'Upper p-value:', pupper
print 'Two-sided p-value:', pboth
print ('\n{0:15} {1:8} {2:8} {3:8}'.format('Category', 'Correlation',\
                                           'Upper p-value', 'Two-sided p-value'))
(t, plow, pupper, pboth, sims) = corr(x = stu_female['overall'], \
                                      y = stu_female['gender_concordance_actual'], seed = rs)
print ('{0:15} {1:8.3f} {2:10.2f} {3:10.2f}'.format('Overall', t, pupper, pboth))

for col in categories:
    (t, plow, pupper, pboth, sims) = corr(x = stu_female[col], \
                                          y = stu_female['gender_concordance_actual'], seed = rs)
    print ('{0:15} {1:8.3f} {2:10.2f} {3:10.2f}'.format(col, t, pupper, pboth))

Male students

Number of male students: 20
Correlation: -0.0718144366713
Upper p-value: 0.6723
Two-sided p-value: 0.7223

Category        Correlation Upper p-value Two-sided p-value
Overall           -0.072       0.68       0.72
professional       0.080       0.36       0.75
respect            0.080       0.44       0.83
caring            -0.106       0.73       0.59
enthusiastic      -0.072       0.57       0.82
communicate       -0.010       0.60       0.84
helpful            0.014       0.51       0.96
feedback          -0.117       0.71       0.69
prompt            -0.049       0.64       0.88
consistent         0.054       0.49       0.85
fair              -0.034       0.63       0.88
responsive        -0.064       0.56       0.84
praised            0.010       0.57       1.00
knowledgeable      0.106       0.42       0.70
clear             -0.119       0.72       0.64

Female students

Number of female students: 23
Correlation: 0.132379460479
Upper p-value: 0.3361
Two-sided p-val

## Grades and instructor gender

### Course grade and reported instructor gender
Do students of male- and female-identified instructors perform equally, as measured by course grade? We do a stratified two-sample permutation t-test, where strata are defined by actual instructor.

In [14]:
(p, t) = stratified_two_sample(grades['grade'][grades.taidgender==1], grades['grade'][grades.taidgender==0], \
                               grades['tagender'][grades.taidgender==1], grades['tagender'][grades.taidgender==0], \
                              stat = 't', alternative = "two-sided", seed = rs)
print 'Course grade and reported instructor gender:'
print 't statistic:', np.round(t, 5)
print 'P-value (two-sided):', np.round(p, 5)
print 'Number of students of male-identified instructors:', np.sum(grades.taidgender==1)
print 'Number of students of female-identified instructors:', np.sum(grades.taidgender==0)

Course grade and reported instructor gender:
t statistic: 0.59964
P-value (two-sided): 0.53839
Number of students of male-identified instructors: 23
Number of students of female-identified instructors: 24


### Course grade and actual instructor gender
Do students of male and female instructors perform equally, as measured by course grade?  We do an unstratified two-sample permutation t-test.

In [15]:
(p, t) = two_sample(grades['grade'][grades.tagender==1], grades['grade'][grades.tagender==0], \
                              stat = 't', alternative = "two-sided", seed = rs)
print 'Course grade:'
print 't statistic:', np.round(t, 5)
print 'P-value (two-sided):', np.round(p, 5)
print 'Number of students of male instructors:', np.sum(grades.taidgender==1)
print 'Number of students of female instructors:', np.sum(grades.taidgender==0)

Course grade:
t statistic: -2.46688
P-value (two-sided): 0.01597
Number of students of male instructors: 23
Number of students of female instructors: 24


In [16]:
np.sum(ratings['gender']==2)

23

## References

MacNell, L., Driscoll, A., and Hunt, A.N., 2015. What’s in a Name: Exposing Gender Bias in Student Ratings of Teaching, _Innovative Higher Education_, _40_, 291&ndash;303.